The basic given value.

In [2]:
import math
from scipy.integrate import quad
import numpy as np
from sympy import sqrt

#Give data
lp = 38.7 #linear power peak kW/m
day = 360 # one cycle duration of 360 days

#Neutronic specifications
phi = 6.1 #neutron flux at peak power node (10*e15 n*cm-2*s-1)

#Thermo-hydraulic specifications
pitch = 8.275e-3 #pin pitch
Co_in_t = 395+273.15 #Coolant inlet temperature K 开尔文
Co_in_p = 0.1 #Coolant inlet pressure (MPa)
Co_m = 0.049 #Coolant mass flow rate(kg s-1)

#Fuel pin specification
F_ch = 850e-3 #Fuel column height m
F_od = 5.42e-3 #Fuel outer diameter m
F_h = 7e-3 #Fuel pellet heigth m
F_td = 11.31e3 #Fuel theretical density kg/m3
F_d = 0.945  #Fuel density %TD
F_gd = 10e-6 #Fuel grain diameter
omr = 1.957 #Oxygen-to-metal ratio(O/M)
Cot = 6.55e-3 #Cladding outer diameter
#Filling gas is Helium
Ipg = 0.1 #Initial pressure of filling gas MPa
Itg = 20 #Initial temperature of filling gas degC 摄氏度

pi = 3.14

# Material composition
#Fuel composition
U = 0.71 # Natural uranium composition
Plu = 0.29 # Given plutonium content

# Coolant Properties
Cp = 1608 - 0.7481*Co_in_t + 3.929e-4*(Co_in_t**2) # Isobaric specific heat
f = 1/2 # 三角形嘅coolant

print(f"Cp = {Cp} ")


# All the temperature is 开尔文 需要+273.15

# To calculate burnup
P_t = 0.0261 # Total power of the 850mm fuel MW
t = 360 # 360days for a cycle
m_f = F_d*F_td * pi*(F_od/2)**2*F_ch# Mass of the fuel
r_m = (238/270)*U+(239/271)*Plu# ratio of the fuel composition massU/massUO2 and Pu

print(f"ratio of the fuel composition : {r_m}")
print(f"mass of the fuel : {m_f} [kg] ")

x = 2 #Deviation from stoichiometery
A = 0.01926 + 1.06e-6*x + 2.63e-8*Plu #Value of A
B = 2.39e-4 + 1.37e-13*Plu  #Value of B
D = 5.27e9 #Value of D
E = 17109.5 #Value of E
Alpha_L = 1.2e-5 # Linear thermal expansion coefficient

# first define an array from bottom to upper part of the fuel
z_ar = np.linspace(-0.425,0.425,100);

# define lower z position
lower_z_limit = z_ar[0];

# try to calculate all the temperature in the same z axis
# create result array for different height
temp_ar = np.zeros(len(z_ar))
error = np.zeros(len(z_ar))
result = np.zeros(len(z_ar))

# define lower z position
lower_z_limit = z_ar[0];
#This is the function of the power distribution
def integrand(x):
    return (-121.78*x**2 + 15.053*x + 37.752)*1000 #[W/m]

#Heat flux distribution unit W/m, slice 100, q prime
hfd = integrand(z_ar)
print(f"heat flux distribution : {hfd}")


for i in range(len(z_ar)):
  result[i], error[i] = quad(integrand, lower_z_limit, z_ar[i]);
  temp_ar[i] = Co_in_t + f*result[i]/(Co_m*Cp)-273.15;

print(f"middle temperature of the coolant : {temp_ar[int(len(z_ar)/2)]} [degC]")
print(f"outlet temperature of the coolant : {temp_ar[-1]} [degC]")
print(temp_ar[0])


Cp = 1283.55714060025 
ratio of the fuel composition : 0.8816083094164275
mass of the fuel : 0.209498814125055 [kg] 
heat flux distribution : [ 9357.9625     10366.97615677 11358.0353089  12331.13995638
 13286.29009922 14223.48573742 15142.72687098 16044.0134999
 16927.34562417 17792.7232438  18640.14635879 19469.61496914
 20281.12907484 21074.6886759  21850.29377232 22607.9443641
 23347.64045123 24069.38203372 24773.16911157 25459.00168478
 26126.87975334 26776.80331726 27408.77237654 28022.78693118
 28618.84698118 29196.95252653 29757.10356724 30299.30010331
 30823.54213473 31329.82966151 31818.16268365 32288.54120115
 32740.96521401 33175.43472222 33591.94972579 33990.51022472
 34371.11621901 34733.76770865 35078.46469365 35405.20717401
 35713.99514973 36004.8286208  36277.70758724 36532.63204903
 36769.60200617 36988.61745868 37189.67840654 37372.78484976
 37537.93678834 37685.13422227 37814.37715157 37925.66557622
 38018.99949622 38094.37891159 38151.80382231 38191.2742284
 38212.

The coolant and the cladding heat transfer part

In [3]:
from sympy import sqrt
import math

# coolant properties
# density #density of coolant #kg m-3 but we use the 华氏度 Fahrenhei
temp_ar_F = temp_ar * 9/5 +32;

def den_cool(x):
  return 954.1579 + x* (x * (x * (0.9667e-9) - 0.46e-5) -0.1273534);

# isobaric specific heat J Kg-1 K-1
temp_ar_K = temp_ar + 273.15;
def Cp(y):
  return 1608 - 0.7481*y + 3.929e-4*(y**2);
## Thermal conductivity of coolant #W m-1 K-1
def TC_cool(z):
  return 110 - 0.0648* z + 1.16e-5*(z**2)

#dynamic viscosity
def eta(a):
 return (np.exp(813.9/a - 2.530))*0.001 #eta Dynamic viscosity [mPa s]
#print(eta(temp_ar_K))

# prandtl number
Pr = Cp(temp_ar_K) * eta(temp_ar_K) / TC_cool(temp_ar_K); # Prandlt number
#print(Pr)

# flow velocity of coolant m/s
# coolant of triangle 三角形的流体冷却剂 三角形减去三个 60度的扇形
sqrt_3 = np.sqrt(3)
A = (sqrt_3)/4 * pitch**2 - 3*(60/360) * (Cot/2)**2 * np.pi
def vf(b):
  return Co_m * 1/den_cool(temp_ar_K) / A
#print(vf(temp_ar_K))

#Reynolds number
Re = den_cool(temp_ar_F) * vf(temp_ar_K) * (Cot/2) / eta(temp_ar_K)
#print(Re)

# Pe Peclet number
Pe = Re * Pr
#print("peclet number :" ,Pe)

# Nu Nusselt number
Nu = 7 + 0.025*Pe**0.8  # 努塞尔数
#print(Nu)

# We go on to calculate the cladding temperature, we need to find the h Convective Heat Transfer Coefficient
#得到了努塞尔数之后我们倒推对流换热系数
Dh = np.sqrt(3)*(pitch)- np.pi/2 * (Cot**2) / pitch # Hydraulic diameter
h = Nu * TC_cool(temp_ar_K) / Dh
#print(f"convective heat transfer coefficient : {h} W/m2 K")

#And we can calculate the heat flux q'', we can find the heat flux distribution hfd from pervious code
# q prime prime is the heat flux in cladding surface, slice for 100, and the unit is W/m2 这里得到的是表面的热通量或者说是热流
qpp = hfd / (np.pi * Cot ) #(10**6)) now it shoudl be the W/m2
#print(f"q prime prime, heat flux distribution in the cladding surface : {qpp} MW/m2")

#Then we use the function with the h 用对流换热方程做, and we can get the cladding outer surface temperature
Tcot = qpp/h + temp_ar # notice T_c cladding T_co coolant  unit is deC
print(f"cladding outer middle temperature : {Tcot} [degC]")
print(f"cladding outer temperature : {Tcot[50]} [degC]")

#Cladding properties 包壳的参数
#using the formular of the the sodium heat transfer
Tcotk = Tcot + 273.15 # transfer to Kelvins 换开尔文

#cladding thermal conductivity W m-1 K-1 开尔文
TC_clad = 13.95 + 0.01163*Tcot
#print(f" cladding thermal conductivity : {TC_clad} W m-1 K-1")

# From the lecture we know the cladding thickness of fast reactor should be from 0.3mm to 0.6mm
# The thickness assumption
ct = ((0.6+0.3)*10**-3)/2# ct is cladding thickness m 0.00045m
#print(f"cladding thickness : {ct} m")

#Cladding inner diameter m
Cid = Cot - ct #cladding inner diameter m
#print(f"cladding inner diameter : {Cid} m")


# Total power for each point, the slice is 100
P = hfd * F_ch # unit is W/m
#print(f"total power for each point : {P} W/m")

# heat transfer function of the sodium cladding
Tcit = Tcot + P * (np.log((Cot/2)/(Cid/2)))/(2 * np.pi * F_ch * TC_clad)
print(f"cladding inner middle temperature : {Tcit[50]} [degC]")
print(f"cladding inner temperature : {Tcit} [degC]")

cladding outer middle temperature : [399.40984693 400.56065321 401.7718755  403.04233219 404.37083826
 405.75620539 407.19724195 408.69275315 410.24154103 411.84240456
 413.49413968 415.19553939 416.94539377 418.74249005 420.58561269
 422.4735434  424.40506123 426.37894259 428.39396132 430.44888876
 432.54249378 434.67354284 436.84080006 439.04302724 441.27898393
 443.5474275  445.84711316 448.17679402 450.53522116 452.92114369
 455.33330874 457.77046159 460.23134568 462.71470267 465.2192725
 467.74379344 470.28700213 472.84763367 475.42442162 478.01609812
 480.6213939  483.23903835 485.86775957 488.50628443 491.15333865
 493.80764682 496.46793248 499.1329182  501.8013256  504.47187542
 507.1432876  509.81428135 512.48357518 515.14988698 517.81193409
 520.46843337 523.11810124 525.75965378 528.39180678 531.0132758
 533.62277625 536.21902349 538.80073284 541.36661969 543.91539958
 546.44578825 548.95650173 551.44625639 553.91376906 556.35775707
 558.77693833 561.17003143 563.53575571 56

Gas gap(Helium) heat transfer calculation


In [4]:
# GAS gap of the fuel Helium 中间填满了氦气，现在计算气体部分
# A of the helium
A = 15.8 # 15.8

Tmiddlecladin = Tcit[50]
# gap temperature
# the gap thermal resistance is defined as
# h_gas = k_gas / t_eff; R = 1/(2*pi*R_g*h_g); delta_T = q*R
# we assume the cladding inner t is equal to the gas stater t
# gas conductivity equation
def k_gas(Tcit): # Tcit = Tgo temperature of the outer gas which is the gas of the inner cladding surface
  return A*10**-4 * Tcit**0.79;

# since the conductivity is not constant and temperature is varied a lot, we can only use iteration to find the fuel outer temperature
t_eff =  (Cid - F_od)/2  # [m] effective thickness radius of the gap
t_eff_ar = np.linspace(0,t_eff,100); # [m]
#MOX_radii_out = MOX_dia/2; # [m] # the fuel outer diameter
gap_ar = np.flip(t_eff_ar)*2+ F_od; # [m] + fuel outer diameter
# equation is q" = delta T / (t/k)
# create gas temperature array
temp_gas = np.zeros(len(gap_ar));

for x in range(len(temp_gas)):
  if x == 0:
    temp_gas[x]=Tmiddlecladin + 273.15; # assign T(x=0)=cladding_in_0 [K] the bottom of the cladding is x =0
  else:
    temp_gas[x]=F_od/(2*np.pi)*((t_eff_ar[x]-t_eff_ar[x-1])/ F_od )*(1/k_gas(temp_gas[x-1]))+temp_gas[x-1];

print(f"temperature of the gas: {temp_gas}")

# convert the temperature to degC
temp_gas_degC = temp_gas-273.15; # [degC]
fuel_temp_out = temp_gas_degC[-1];
print('fuel outer temperature:',fuel_temp_out, '[degC]')

print(temp_gas_degC)

# fuel is conduction which at center is highest temperature
# I = q'/(4*pi) = k(T) * delta_T
# I is a curve with unknow function but we can use estimation
# m /aprrox 1/50, means 50 degC increment cause 1 W/cm
# delta_T=\frac{q'}{4*pi*k}
# and LHT is in meter and unit need to be convert to cm
#LHT_cm = LHT/100; # [W/cm]
LHT_cm = F_od/100; # [W/cm]
I = LHT_cm/(4 * np.pi);
# Linearize the I
Int_to_temp = 50 * I; # [degC]

fuel_temp_max = fuel_temp_out + Int_to_temp; # [degC]
print('fuel maximum temperature:',fuel_temp_max, '[degC]')

# plot graph at z=0




temperature of the gas: [801.87539931 801.87540106 801.87540282 801.87540458 801.87540633
 801.87540809 801.87540985 801.8754116  801.87541336 801.87541512
 801.87541688 801.87541863 801.87542039 801.87542215 801.8754239
 801.87542566 801.87542742 801.87542917 801.87543093 801.87543269
 801.87543444 801.8754362  801.87543796 801.87543972 801.87544147
 801.87544323 801.87544499 801.87544674 801.8754485  801.87545026
 801.87545201 801.87545377 801.87545553 801.87545729 801.87545904
 801.8754608  801.87546256 801.87546431 801.87546607 801.87546783
 801.87546958 801.87547134 801.8754731  801.87547485 801.87547661
 801.87547837 801.87548013 801.87548188 801.87548364 801.8754854
 801.87548715 801.87548891 801.87549067 801.87549242 801.87549418
 801.87549594 801.8754977  801.87549945 801.87550121 801.87550297
 801.87550472 801.87550648 801.87550824 801.87550999 801.87551175
 801.87551351 801.87551527 801.87551702 801.87551878 801.87552054
 801.87552229 801.87552405 801.87552581 801.87552756 8

In [ ]:


Beta = P_t*t/(m_f*r_m) #Burnup GWd/t
print(f"Burnup value is : {Beta} [MWd/kg]" )

#Melting temperature of MOX K 开尔文
T_m = 2964.92 + ((3147-364.85*Plu-1014.15*x)-2964.92)*math.exp(-Beta/41.01)
print(f"melting temperature: {T_m} [K]")
#Thermal conductivity MOX W/m*K
#k = 1.755 + (k0-1.755)*math.exp(-Beta/128.75)

#k0 = (1/(A+B*T_k)+D/T_ke2*math.exp(-E/T_k))*(1-p)**2.5

T_c = Co_in_t + f*37.752/(Co_m*Cp)   # Temperature of the coolant 在正中间的冷却剂温度
print (f"middle temperature of the coolant : {T_m} [K]")


# We go on to calculate the cladding temperature, we need to find the h Convective Heat Transfer Coefficient

#coolant properties
TF_c = T_c*9/5 +32 # 华氏度 Fahrenheit center of the cladding and coolant
Dh = (1/(pitch**2)) * ((1/(3**2)*(pitch**2)- pi/2 * (Cot**2)))   # Hydraulic diameter
D_c = 954.1579 + TF_c* (TF_c * (TF_c * (0.9667e-9) - 0.46e-5) -0.1273534)  #Coolant density
TC = 110 - 0.0648* T_c + 3.929e-4*(T_c**2) # Thermal conductivity

def Reynolds
 return D_c * velocity * Dh / eta


eta = math.exp(813.9/T_c - 2.530) #eta Dynamic viscosity [mPa s]
Re = D_c * velocity * Dh / eta #雷诺数
Pr = Cp * eta / TC # Prandlt number

Pe = Re * Pr
Nu = 7 + 0.025*Pe**0.8  # 努塞尔数



In [ ]:
# difference btw lambda and def

def sum(a,b):
  return a+b;
y=sum(1,2);
print(y)

x = lambda c,d: c+d
q=x(1,2)
print(q)

3
3


In [ ]:
from scipy.integrate import quad
import numpy as np

# first define an array from bottom to upper part of the fuel
z_ar = np.linspace(-0.425,0.425,100);

# define lower z position
lower_z_limit = z_ar[0];

# Ayip code
# define the square polynomical to fit the power factor data
def integrand(x):
    return (-121.78*x**2 + 15.053*x + 37.752)*1000 #[W/m]

# do the intergral to get the result and the error from integrand method from lowest point to middle point
result_mid, error_mid = quad(integrand, lower_z_limit, z_ar[int(len(z_ar)/2)]);

# estiamte the temperature of from inlet to the position variable "z"
T_mid = Co_in_t + f*result_mid/(Co_m*Cp); # [degC]
print(f"middle temperature of the coolant : {T_mid-273.15} [degC]")

# do the intergral to get the result and the error from integrand method from lowest point to top point
result_top, error_top = quad(integrand, lower_z_limit,z_ar[-1]);

# estiamte the temperature of from inlet to the position variable "z"
T_top = Co_in_t + f*result_top/(Co_m*Cp); # [degC]
print(f"outlet temperature of the coolant : {T_top-273.15} [degC]")

middle temperature of the coolant : 488.2608798949759 [degC]
outlet temperature of the coolant : 600.558014296455 [degC]


In [ ]:
from scipy.integrate import quad
import numpy as np

# try to calculate all the temperature in the same z axis
# create result array for different height
temp_ar = np.zeros(len(z_ar))
error = np.zeros(len(z_ar))
result = np.zeros(len(z_ar))

# define lower z position
lower_z_limit = z_ar[0];

def integrand(x):
    return (-121.78*x**2 + 15.053*x + 37.752)*1000 #[W/m]

for i in range(len(z_ar)):
  result[i], error[i] = quad(integrand, lower_z_limit, z_ar[i]);
  temp_ar[i] = Co_in_t + f*result[i]/(Co_m*Cp)-273.15;

print(f"middle temperature of the coolant : {temp_ar[int(len(z_ar)/2)]} [degC]")
print(f"outlet temperature of the coolant : {temp_ar[-1]} [degC]")
print(f"all temperature profile:" {temp_ar})

middle temperature of the coolant : 488.2608798949759 [degC]
outlet temperature of the coolant : 600.558014296455 [degC]
